In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
df

In [ ]:
test = pd.read_csv('../input/ventilator-pressure-prediction/test.csv')
test

In [ ]:
df = df.astype('int32')

In [ ]:
#aggregate data with breath_id by taking mean

d = df.groupby('breath_id').agg('mean').reset_index()

df = d.sample(frac=1)
y = df['pressure']
X = df.drop(columns=['pressure', 'breath_id'], axis=0 )

X['k']= (X['R']+X['C']+X['u_in']+X['u_out']+X['time_step'])/5
X['M'] = X['u_in']*X['u_out']
X['A'] = X['u_in']+X['u_out']
# X['dd'] = X['u_in']/X['u_out']
X['RC_sum'] = X['R'] + X['C']
# X['RC_div'] = X['R'] / X['C']
X['RC_mul'] = X['R'] * X['C']

# for i in range(50):
#     X['time_lag'+str(i)] = X['time_step'].shift(i).fillna(0)
#     X['u_in_lag'+str(i)] = X['u_in'].shift(i).fillna(0)
#     X['u_out_lag'+str(i)] = X['u_out'].shift(i).fillna(0)


In [ ]:
X = pd.get_dummies(X)

In [ ]:
from sklearn.preprocessing import RobustScaler, MinMaxScaler
scale = MinMaxScaler()
X = scale.fit_transform(X)

In [ ]:
X.shape

In [ ]:
# train with the same configuration as below
from tensorflow.keras.layers import Dense, LSTM, GRU, Dropout, Flatten, Bidirectional, Conv1D, MaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Dropout, Flatten, AveragePooling1D
import tensorflow as tf
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import LearningRateScheduler

tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

def baseline_model():
    with tpu_strategy.scope():
        model = Sequential()
        model.add(Conv1D(filters=32, kernel_size=1, activation='tanh',padding="causal", input_shape=(1, 11)))
        model.add(Conv1D(filters=64, kernel_size=1, activation='tanh'))
        model.add(Bidirectional(GRU(128, kernel_initializer='normal', return_sequences=True,activation='relu')))
        model.add(Bidirectional(GRU(1024, kernel_initializer='normal', return_sequences=True,activation='relu')))
        model.add(Bidirectional(GRU(64, kernel_initializer='normal', return_sequences=True,activation='selu')))
        model.add(Flatten())
        model.add(Dense(32, kernel_initializer='normal', activation='selu'))
        model.add(Dense(16, kernel_initializer='normal', activation='selu'))
        model.add(Dense(1,kernel_initializer='normal'))
        model.compile(loss='mean_absolute_error', optimizer='adam')
        return model


model = baseline_model()

scheduler = ExponentialDecay(1e-3, 400*((len(X)*0.8)/512), 1e-5)
lr = LearningRateScheduler(scheduler, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience = 3, restore_best_weights=True )
history = model.fit(np.expand_dims(X, axis=1),y,validation_split=0.2, epochs=50, callbacks=[lr])

In [ ]:
test = test.astype('int32')
test = test.drop(columns = ['breath_id'], axis=0)
test['k']= (test['R']+test['C']+test['u_in']+test['u_out']+test['time_step'])/5
test['M'] = test['u_in']*test['u_out']
test['A'] = test['u_in']+test['u_out']
test['RC_sum'] = test['R'] + test['C']
test['RC_mul'] = test['R'] * test['C']
tes = pd.get_dummies(test)
tes = scale.fit_transform(tes)

test_pred = model.predict(np.expand_dims(tes, axis=1))

In [ ]:
test_pred

In [ ]:
df = pd.DataFrame(data = test_pred, index = test['id'], columns = ['pressure'])
df.to_csv('./submission.csv', header=True)